Loading the packages

In [1]:
import sys
print(sys.executable)

c:\Users\sknecht\Projects\databricks-mazda\.venv\Scripts\python.exe


In [3]:
%%python -m ipython kernel install --user --name=.venv

UsageError: %%python is a cell magic, but the cell body is empty.


In [4]:
## Importing the relevant packages

import oracledb
# cx_Oracle.init_oracle_client(lib_dir=r"C:\oracle\instantclient_21_3")
import getpass
import sqlite3
import time

import pandas as pd
import numpy as np
import os
import glob
import openpyxl
import psutil
from datetime import timedelta
from datetime import datetime


import re
import datetime
import gc

import matplotlib.pyplot as plt; plt.rcdefaults()
import matplotlib.pyplot as plt

from itertools import islice 


## Let the display show more rows & columns
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 100)
pd.set_option('display.expand_frame_repr', True)


C:\Users\sknecht\AppData\Local\Temp\ipykernel_21524\337264456.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
environment = input('environment (DVL, TST, ACP, PRD): ')
username = input('username: ')
password = getpass.getpass('password: ')

In [6]:
def connectionpool(i):
    switcher={
        'DVL':'dvldw-scan.mle.mazdaeur.com:1521/DVL_DWH',
        'TST':'tstdw-scan.mle.mazdaeur.com:1521/TST_DWH',
        'ACP':'accdw-scan.mle.mazdaeur.com:1521/ACP_DWH',
        'PRD':'prddw-scan.mle.mazdaeur.com/PRD_DWH'        
    }
    return switcher.get(i,"No valid connection name")

In [7]:
connectionpool(environment)

'prddw-scan.mle.mazdaeur.com/PRD_DWH'

In [8]:
# Connect to the Oracle DB (DVL)
connection = username + "/" + "password:" + connectionpool(environment)
con = oracledb.connect(user=username, password=password, port=1521, dsn=connectionpool(environment))

print(connection)

DWSRV/password:prddw-scan.mle.mazdaeur.com/PRD_DWH


In [9]:
# Now execute the sqlquery 
cursor = con.cursor() 

In [10]:
cursor.arraysize = 10000

In [11]:
# execute SQL 
cursor.execute("select 'connection established' from dual") 
for row in cursor: print(row)

('connection established',)


In [ ]:
## market selection
## GER = '1-GZI'
## UK = '1-GZE'


### UK

In [ ]:
tcmr_base=None
dlr_base=None
sales_base=None

In [22]:
### Loading TCMR data UK
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-GZE'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\3776240913.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:543.2142488956451


In [23]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MMUK'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\4185893535.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


sql execute time:0.22063851356506348


In [24]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-GZE' or dcc.tech_bu_id = '1-GZE')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\4051197299.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:153.91302728652954


In [25]:
import os
path = r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\'
os.makedirs(path, exist_ok=True)

In [28]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\UKtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\UKdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\UKsales_base.csv', index = False, encoding = 'utf-8-sig')

### Germany

In [ ]:
tcmr_base=None
dlr_base=None
sales_base=None

In [29]:
### Loading TCMR data Ger
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-GZI'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\2819601676.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:821.0267267227173


In [30]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MMD'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\420099745.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


sql execute time:0.41649580001831055


In [31]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-GZI' or dcc.tech_bu_id = '1-GZI')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\3909614525.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:265.6879403591156


In [32]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\GERtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\GERdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\GERsales_base.csv', index = False, encoding = 'utf-8-sig')

### Switzerland

In [ ]:
tcmr_base=None
dlr_base=None
sales_base=None

In [33]:
### Loading TCMR data Ger
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-8PMCW'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\4264391750.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:68.2197413444519


In [34]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MS'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

sql execute time:0.1143791675567627


C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\893207345.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


In [35]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-8PMCW' or dcc.tech_bu_id = '1-8PMCW')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\457636194.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:45.01234269142151


In [36]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\CHtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\CHdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\CHsales_base.csv', index = False, encoding = 'utf-8-sig')

### Spain

In [ ]:
tcmr_base=None
dlr_base=None
sales_base=None

In [37]:
### Loading TCMR data Spain
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-8PMAJ'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\3309356607.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:139.6901924610138


In [38]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MAE'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\1464237243.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


sql execute time:0.2583811283111572


In [39]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-8PMAJ' or dcc.tech_bu_id = '1-8PMAJ')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\495616538.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:77.11185908317566


In [40]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\EStcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ESdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ESsales_base.csv', index = False, encoding = 'utf-8-sig')

### Italy

In [ ]:
tcmr_base=None
dlr_base=None
sales_base=None

In [41]:
### Loading TCMR data Spain
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-8PMDL'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\323868668.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:67.42658352851868


In [42]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MMI'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

sql execute time:0.13547229766845703


C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\120558498.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


In [43]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-8PMDL' or dcc.tech_bu_id = '1-8PMDL')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\4143542672.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:92.92764925956726


In [44]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ITtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ITdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ITsales_base.csv', index = False, encoding = 'utf-8-sig')

#### France

In [ ]:
tcmr_base=None
dlr_base=None
sales_base=None

In [45]:
### Loading TCMR data France
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-8PM8P'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\1548607196.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:82.64949107170105


In [46]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MAF'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\2620639876.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


sql execute time:4.609524250030518


In [47]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-8PM8P' or dcc.tech_bu_id = '1-8PM8P')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\3179698137.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:369.835116147995


In [48]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\FRtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\FRdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\FRsales_base.csv', index = False, encoding = 'utf-8-sig')

### Poland

In [12]:
tcmr_base=None
dlr_base=None
sales_base=None

In [49]:
### Loading TCMR data France
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-8PMCM'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\2529471146.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:91.48964095115662


In [50]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MMPOL'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

sql execute time:0.12750768661499023


C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\202411213.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


In [51]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-8PMCM' or dcc.tech_bu_id = '1-8PMCM')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\951546004.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:84.39636039733887


In [52]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\POLtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\POLdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\POLsales_base.csv', index = False, encoding = 'utf-8-sig')

# Austria

In [ ]:
tcmr_base=None
dlr_base=None
sales_base=None

In [53]:
### Loading TCMR data Austria
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-8PMCM'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\3530738084.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:80.47776365280151


In [54]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MAG'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\3799364023.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


sql execute time:0.2149968147277832


In [55]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-8PMCM' or dcc.tech_bu_id = '1-8PMCM')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_28764\951546004.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:50.45028471946716


In [56]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ATtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ATdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\ATsales_base.csv', index = False, encoding = 'utf-8-sig')

# Ireland

In [13]:
tcmr_base=None
dlr_base=None
sales_base=None

In [14]:
### Loading TCMR data Ireland
start_time = time.time()

tcmr_base=pd.read_sql(
    '''select 
    dtc.agree_num as "ref_num",
    ftph.updated_sbl as "tcmr_updated",
    ftph.status as "tcmr_pw_status",
    ftph.renewal_type as "tcmr_type",
    ftph.tcmr_lead_stat as "tcmr_lead",
    ftph.payment_walk as "payment_walk",
    ddl.dlr_code as "crnt_dlr_code",
    ddl.dlr_name as "crnt_dlr_name"
from dwh.fact_tcmr_contract ftc
    join dwh.dim_tcmr_contract dtc
        on ftc.tcmr_contract_sk = dtc.tcmr_contract_sk
    left join dwh.FACT_TCMR_PAYMENT_HIST ftph
        on dtc.tcmr_contract_sk = ftph.tcmr_contract_sk
    left join dwh.dim_tcmr_contract dtc
        on ftph.tcmr_contract_sk = dtc.tcmr_contract_sk
    join dwh.DIM_DLR ddl
        on ftc.dlr_sk = ddl.dlr_sk
where
    1 = 1
    and ftc.tech_bu_id = '1-8PMCM'
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

tcmr_base = tcmr_base.sort_values('tcmr_updated', ascending=False).drop_duplicates('ref_num').sort_index()

C:\Users\sknecht\AppData\Local\Temp\ipykernel_21524\2187602347.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tcmr_base=pd.read_sql(


sql execute time:65.1290111541748


In [15]:
### Loading dlr data
start_time = time.time()

dlr_base=pd.read_sql('''select 
    ddl.dlr_code as "dlr_code",
    ddl.dlr_name as "dlr_name",
    ddl.dlr_status as "dlr_status",
    ddl.dlr_sales_district as "mazda_zone",
    ddl.dlr_bank_district as "bank_zone",
    ddl.DLR_SALES_ALLOW_ORDER_FLAG as "sales_flag"
from dwh.DIM_DLR ddl
where 
    1 = 1
    and ddl.dlr_dist_code = 'MMIRL'
    and ddl.dlr_sales_district is not null
''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

sql execute time:0.07500386238098145


C:\Users\sknecht\AppData\Local\Temp\ipykernel_21524\1909522352.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dlr_base=pd.read_sql('''select


In [16]:
#download sales data

start_time = time.time()

sales_base=pd.read_sql('''select 
    dcc.con_id as "Contact Row ID",
    fcs.tech_accnt_id as "Account Row ID",
    dcc.first_name as "First Name",
    dcc.last_name as "Last Name",
    dcc.date_of_birth as "Birthday",
    dcc.address_line_1 as "Street Address",
    dcc.city as "City",
    dcc.post_code as "Postal Code",
    dcc.country as "Country",
    dcc.phone_private as "Private Phone #",
    dcc.phone_work as "Business Phone #",
    dcc.phone_mobile as "Cell Phone #",
    dcc.email_private as "Private Email",
    dcc.email_work as "Business Email",
    dca.account_name as "Account Name",
    dca.addr as "Street Address.1",
    dca.email_1 as "BBusiness Email",
    dca.email_2 as "BPrivate Email",
    dca.city as "City.1",
    dca.zipcode as "Postal Code.1",
    dca.county as "County.1",
    dlr.dlr_code as "Dealer #",
    dlr.dlr_name as "Dealer Name",
    dlr.dlr_country_name as "Dealer_Country_I",
    fcs.tech_sale_dt as "Sale Date",
    fcs.tech_vin_cd as "VIN",
    dc.lcns_num as "Registration #",
    dc.registered_dt as "Registration Date",
    dc.model_cd as "Model_I",
    dvm.veh_mdl_product_code as "Model Number",
    dcs.sale_class_cd as "New/Used/Demo",
    dc.fuel_cd as "Fuel"
from dwh.fact_car_sales fcs
    join dwh.dim_car_sales dcs
        on fcs.car_sale_sk = dcs.car_sale_sk
    join dwh.dim_dist dd
        on fcs.dist_sk = dd.dist_sk
    join dwh.dim_car dc
        on fcs.car_sk = dc.car_sk
    join dwh.DIM_DLR dlr
        on fcs.dlr_sk = dlr.dlr_sk
    join dwh.DIM_VEH_MODEL dvm
        on dvm.veh_mdl_sk =  dc.veh_mdl_sk
    join dwh.dim_customer_contact dcc
        on fcs.customer_contact_owner_sk = dcc.customer_contact_sk
    join dwh.dim_customer_account dca
        on fcs.customer_account_sk = dca.customer_account_sk
    where 
    1 = 1
    and (dca.tech_bu_id = '1-8PMCM' or dcc.tech_bu_id = '1-8PMCM')
    and dc.make_cd = 'Mazda'
    and fcs.cal_sale_sk >= 20140601
    and dc.serial_num is not null
    and dcs.sale_class_cd is not null
    and (dcc.con_id is not null or fcs.tech_accnt_id is not null)
    ''', con)

end_time = time.time()
print("sql execute time:%s" % (end_time - start_time))

C:\Users\sknecht\AppData\Local\Temp\ipykernel_21524\951546004.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sales_base=pd.read_sql('''select


sql execute time:42.28795504570007


In [17]:
tcmr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\IRLtcmr_base.csv', index = False, encoding = 'utf-8-sig')
dlr_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\IRLdlr_base.csv', index = False, encoding = 'utf-8-sig')
sales_base.to_csv(r'C:\\Data Analytics\\Renewals Dashboard - Sales data\\IRLsales_base.csv', index = False, encoding = 'utf-8-sig')

In [61]:
print('ALL Done - great job!')

ALL Done - great job!


In [62]:
## Upload and replace existing files in Teams: MLE_P_IT_Power BI rollout/Pilot - MME Financial Services/renewal report input data